## HS.1.1

In [ ]:
dev = qml.device("default.qubit", wires=1)

def exact_result_XandZ(alpha, beta, time):
    """Exact circuit for evolving a qubit with H = alpha Z + beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        
    Returns: 
        array[complex]: The exact state after evolution.
    """
    root = np.sqrt(alpha**2 + beta**2)
    c_0 = np.cos(root*time) - (alpha/root)*np.sin(root*time)*1.j
    c_1 = -(beta/root)*np.sin(root*time)*1.j
    return np.array([c_0, c_1])
    
@qml.qnode(dev)
def trotter_XandZ(alpha, beta, time, n):
    """Trotterized circuit for evolving a qubit with H = alpha Z + beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        array[complex]: The state after applying the Trotterized circuit.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    for _ in range(n):
        qml.RZ(2*alpha*time/n, wires=0)
        qml.RX(2*beta*time/n, wires=0)

    return qml.state()

def trotter_error_XandZ(alpha, beta, time, n):
    """Difference between the exact and Trotterized result.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        float: The distance between the exact and Trotterized result.
    """
    diff = np.abs(trotter_XandZ(alpha, beta, time, n) - exact_result_XandZ(alpha, beta, time))
    return np.sqrt(sum(diff*diff))



## HS.1.2

In [ ]:
@qml.qnode(dev)
def trotter_2_XandZ(alpha, beta, time, n):
    """Second-order Trotter circuit for the Hamiltonian H = alpha Z +  beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        array[complex]: The state after applying the second-order circuit.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    for _ in range(n):
        qml.RZ(alpha*time/n, wires=0)
        qml.RX(beta*time/n, wires=0)
        qml.RX(beta*time/n, wires=0)
        qml.RZ(alpha*time/n, wires=0)

    return qml.state()

def trotter_2_error_XandZ(alpha, beta, time, n):
    """Difference between the exact and second-order Trotter result.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        
    Returns: 
        float: The distance between the exact and second-order result.
    """
    diff = np.abs(trotter_2_XandZ(alpha, beta, time, n) - exact_result_XandZ(alpha, beta, time))
    return np.sqrt(sum(diff*diff))

## HS.1.3

In [ ]:
@qml.qnode(dev)
def trotter_k_XandZ(alpha, beta, time, n, k):
    """
    Order-2k Trotter circuit for the Hamiltonian H = alpha Z + beta X.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        k (int): The order of our Trotterization formula divided by 2.
        
    Returns: 
        array[complex]: The state after applying the order-2k circuit.
    """
    def U(alpha, beta, time, n, k):
        if k == 1:
            qml.RZ(alpha*time/n, wires=[0])
            qml.RX(2*beta*time/n, wires=[0])
            qml.RZ(alpha*time/n, wires=[0])
        else:
            ##################
            # YOUR CODE HERE #
            ##################
            s2k = 1 / (4 - 4 ** (1/(2 * k - 1)))
            U(alpha, beta, s2k * time, n, k - 1)
            U(alpha, beta, s2k * time, n, k - 1)
            U(alpha, beta, (1 - 4 * s2k) * time, n, k - 1)
            U(alpha, beta, s2k * time, n, k - 1)
            U(alpha, beta, s2k * time, n, k - 1)
    for _ in range(n):
        U(alpha, beta, time, n, k)
    return qml.state()

def trotter_k_error_XandZ(alpha, beta, time, n, k):
    """
    Difference between the exact and order-2k Trotter result.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        n (int): The number of steps in our Trotterization.
        k (int): The order of our Trotterization formula divided by 2.
        
    Returns: 
        float: The distance between the exact and order-2k result.
    """
    diff = np.abs(trotter_k_XandZ(alpha, beta, time, n, k) - exact_result_XandZ(alpha, beta, time))
    return np.sqrt(sum(diff*diff))


## HS.1.4

In [ ]:
def trotter_steps_XandZ(alpha, beta, time, error, k):
    """
    Computes the number of Trotter steps needed for a given order k and error.
    
    Args:
        alpha (float): The coefficient of Z in the Hamiltonian.
        beta (float): The coefficient of X in the Hamiltonian.
        time (float): The time we evolve the state for.
        error (float): The size of the tolerated simulation error.
        k (int): The order of our Trotterization formula divided by 2.
        
    Returns: 
        int: The number of steps needed to achieve a given error.
    """
    n = 1
    ##################
    # YOUR CODE HERE #
    ##################
    while trotter_k_error_XandZ(alpha, beta, time, n, k) > 1e-6:
        n += 1
    return n

error = 1e-6
optimal_k = 3 # MODIFY THIS AFTER LOOKING AT THE PLOT 
n = trotter_steps_XandZ(1, 1, 1, error, optimal_k)
depth = qml.specs(trotter_k_XandZ)(1, 1, 1, n, optimal_k)['resources'].depth
print("The Trotter circuit of order", 2*optimal_k, "uses a circuit of depth", depth, "gates to achieve error ε =", error, ".")

## HS.1.5

In [ ]:
time = 1.6

def create_hamiltonian(alpha,beta):
    """
    Generates the Hamiltonian given in the statement
    Args:
        alpha (float): The first parameter of the Hamiltonian
        beta (float): The second parameter of the Hamiltonian
    Returns:
        (qml.Operator): A PennyLane Hamiltonian.
    """

    # Create the Hamiltonian as a function of alpha and beta#

    ######################
    ### YOUR CODE HERE ###
    ######################
    H = qml.Hamiltonian([alpha, beta], [qml.X(0) @ qml.X(1), qml.Z(0) @ qml.Z(1)])

    return H # Return your Hamiltonian

dev = qml.device("default.qubit", wires = 2)

@qml.qnode(dev)
def trotter_evolve(alpha, beta, k, state):
    """
    Returns the Trotter approximation of the time evolution of a given state
    Args:
        alpha: (float) The first parameter of the Hamiltonian
        beta: (float) The second parameter of the Hamiltonian
        k: (int) The order of the Trotter approximation
        state: (numpy.array) A two-qubit state.
    Returns:
    (numpy.array): The evolved state.
    """

    qml.StatePrep(state, wires = range(2))

    ######################
    ### YOUR CODE HERE ###
    ######################
    H = create_hamiltonian(alpha, beta)
    qml.TrotterProduct(H, time, k)
    return qml.state()

alpha = 1
beta = 2
k = 2
state = [1/2,1/2,1/2,1/2]

print("For the chosen alpha, beta, k, and state, the final state is: ", trotter_evolve(alpha, beta, k, state))
